In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

The main aim here is to get confusion metrics and also figure out why each prediction is wrong

## Get accuracy metrics

In [2]:
# single stage combined
# pred_ann_dir = f'../resnet_model/work_dirs/singletask_resnet_fpn_for_verb_and_target_best/combine_first_and_second_stage_results' 
# gt_ann_dir = f'../data/triplet_segmentation_dataset_v2_second_stage/test/ann_dir'
       
pred_ann_dir = f'../resnet_model/work_dirs/singletask_resnet_fpn_verb_and_target_parallel_fc_predict_on_train/combine_first_and_second_stage_results' 
gt_ann_dir = f'../data/triplet_segmentation_dataset_v2_second_stage/train/ann_dir'       

In [3]:
from utils.general.dataset_variables import TripletSegmentationVariables

VERB_ID_TO_CLASS_DICT = TripletSegmentationVariables.categories['verb']
TARGET_ID_TO_CLASS_DICT = TripletSegmentationVariables.categories['target']
INSTRUMENT_ID_TO_CLASS_DICT = TripletSegmentationVariables.categories['instrument']

possible_verbs = list(VERB_ID_TO_CLASS_DICT.values())
possible_targets = list(TARGET_ID_TO_CLASS_DICT.values())
possible_instruments = list(INSTRUMENT_ID_TO_CLASS_DICT.values())

In [4]:
import os
import json
from statistics import mean

def load_annotations(folder):
    """Load annotations from a folder of JSON files."""
    annotations = {}
    for file_name in os.listdir(folder):
        if file_name.endswith('.json'):
            with open(os.path.join(folder, file_name), 'r') as f:
                data = json.load(f)
                annotations[file_name] = data
    return annotations

def calculate_accuracy(ground_truth_folder, prediction_folder, task, possible_classes):
    """Calculate accuracy for possible classes."""
    ground_truth = load_annotations(ground_truth_folder)
    predictions = load_annotations(prediction_folder)
    
    assert task in ['instrument', 'verb', 'target']
    if task == 'instrument':
        task = 'label' # annotations called labels in annotation format

    total_per_class = {class_name: 0 for class_name in  possible_classes }
    correct_per_class = {class_name: 0 for class_name in  possible_classes }    
    false_positives_per_class = {class_name: 0 for class_name in  possible_classes }
    false_negatives_per_class = {class_name: 0 for class_name in  possible_classes }
    
    missed_files = []

    for file_name, gt_data in ground_truth.items():
        if file_name not in predictions:
            missed_files.append(file_name)
            continue
        
        # i am okay with the overwritting of contours, as it essentially we just need to count one.     
        gt_shapes = { (shape['label'],shape['group_id']): shape for shape in gt_data.get('shapes', [])} 
        pred_shapes = {(shape['label'],shape['group_id']): shape for shape in predictions[file_name].get('shapes', [])}
        
        found_gt_keys = []

        for gt_key, gt_shape in gt_shapes.items(): 
            
            gt_class_name = gt_shape[task]
            
            total_per_class[gt_class_name] += 1 

            found_pred_key = None
            
            for pred_key, pred_shape in pred_shapes.items(): 
                # Calculate verb accuracy
                pred_class_name = pred_shape.get(task)
                if gt_class_name == pred_class_name:
                    correct_per_class[pred_class_name] += 1
                    found_pred_key = pred_key
                    found_gt_keys.append(gt_key)
                    break
                    
            # remove that pred prediction from possibilites        
            if found_pred_key:
                pred_shapes.pop(found_pred_key)
        
        #update the false positives from the pred. 
        for pred_key, pred_shape in pred_shapes.items(): 
            pred_class_name = pred_shape.get(task)
            false_positives_per_class[pred_class_name] += 1
        
        
        # delete the found groundtruth.
        for found_gt_key in found_gt_keys: 
            gt_shapes.pop(found_gt_key)
        
         #update the false negatives from the gt. 
        for gt_key, gt_shape in gt_shapes.items(): 
            gt_class_name = gt_shape.get(task)
            false_negatives_per_class[gt_class_name] += 1
        
                

    accuracy = sum(correct_per_class.values()) / sum(total_per_class.values())  if sum(total_per_class.values()) > 0 else 0
    acc_per_class = {
        class_name: (
            correct_per_class[class_name] / total_per_class[class_name] 
            if total_per_class[class_name] is not None and total_per_class[class_name] > 0 
            else None
        )
        for class_name in possible_classes
    }
    mean_acc = mean(value for value in acc_per_class.values() if value is not None)

    return {
        "accuracy": accuracy,
        "mean_acc":mean_acc,
        "acc_per_class": acc_per_class,
        "missed_files": missed_files,
        "total_per_class": total_per_class,
        "correct_per_class": correct_per_class,
        "false_positives_per_class": false_positives_per_class,
        "false_negatives_per_class": false_negatives_per_class
    }



In [5]:
task = 'target'
if task == 'target': 
    possible_classes = possible_targets
elif  task == 'verb': 
    possible_classes = possible_verbs  
elif  task == 'instruments': 
    possible_classes = possible_instruments      
else: 
    raise ValueError('there is an error in the task')     
accuracy_results = calculate_accuracy(gt_ann_dir, pred_ann_dir, task = task, possible_classes=possible_classes)



In [6]:

print("accuracy:", accuracy_results["accuracy"])
print("mean_acc:", accuracy_results["mean_acc"])
print("acc_per_class:", accuracy_results["acc_per_class"])
print("Missed Files:", accuracy_results["missed_files"])
print("total_per_class:", accuracy_results["total_per_class"])
print("correct_per_class:", accuracy_results["correct_per_class"])
print("false_positives_per_class:", accuracy_results["false_positives_per_class"])
print("false_negatives_per_class:", accuracy_results["false_negatives_per_class"])

accuracy: 0.6927348014304536
mean_acc: 0.31684411826454434
acc_per_class: {'gallbladder': 0.92703452232954, 'cystic_plate': 0.7756024096385542, 'cystic_duct': 0.5029828486204325, 'cystic_artery': 0.07065868263473053, 'cystic_pedicle': 0.017241379310344827, 'blood_vessel': 0.14953271028037382, 'fluid': 0.8939393939393939, 'abdominal_wall_cavity': 0.049079754601226995, 'liver': 0.7937823834196891, 'adhesion': 0.12987012987012986, 'omentum': 0.09149130832570906, 'peritoneum': 0.0, 'gut': 0.0, 'specimen_bag': 0.266304347826087, 'null_target': 0.08514190317195326}
Missed Files: []
total_per_class: {'gallbladder': 10457, 'cystic_plate': 664, 'cystic_duct': 2682, 'cystic_artery': 835, 'cystic_pedicle': 58, 'blood_vessel': 107, 'fluid': 528, 'abdominal_wall_cavity': 163, 'liver': 2895, 'adhesion': 77, 'omentum': 1093, 'peritoneum': 50, 'gut': 77, 'specimen_bag': 368, 'null_target': 1198}
correct_per_class: {'gallbladder': 9694, 'cystic_plate': 515, 'cystic_duct': 1349, 'cystic_artery': 59, 'cy

In [7]:
total_per_class = accuracy_results['total_per_class']
correct_per_class = accuracy_results['correct_per_class']


In [8]:
acc_per_class = {
    class_name: (
        correct_per_class[class_name] / total_per_class[class_name] 
        if total_per_class[class_name] is not None and total_per_class[class_name] > 0 
        else None
    )
    for class_name in possible_targets
}
acc_per_class

{'gallbladder': 0.92703452232954,
 'cystic_plate': 0.7756024096385542,
 'cystic_duct': 0.5029828486204325,
 'cystic_artery': 0.07065868263473053,
 'cystic_pedicle': 0.017241379310344827,
 'blood_vessel': 0.14953271028037382,
 'fluid': 0.8939393939393939,
 'abdominal_wall_cavity': 0.049079754601226995,
 'liver': 0.7937823834196891,
 'adhesion': 0.12987012987012986,
 'omentum': 0.09149130832570906,
 'peritoneum': 0.0,
 'gut': 0.0,
 'specimen_bag': 0.266304347826087,
 'null_target': 0.08514190317195326}